In [1]:
import tensorflow as tf
import keras
import os
from tensorflow.keras.backend import set_session

os.environ['CUDA_VISIBLE_DEVICES'] = ""
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [ ]:
import torch
import torch.nn
import numpy as np
import matlab
import matlab.engine
import h5py

In [ ]:
from chemvae_bopt import chemvae_gpopt_pes

In [ ]:
zinc250k = []
with open('/cluster/sj1/bb_opt/chemical_vae/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv') as f:
    next(f)
    for line in f:
        line = [k.strip() for k in line.strip().split('\t')]
        zinc250k += [line[0], [float(k) for k in line[1:]]]

In [ ]:
from chemvae_keras import vae_utils
from chemvae_keras import mol_utils as mu

In [ ]:
is_cuda = True
directory = '../chemical_vae/models/zinc_properties'
n_init_samples = 1000
n_opt_samples = 10
cur_dir = '/cluster/sj1/bb_opt/src'
score_fn = lambda x : x[0]

In [ ]:
def obj(x):
    noise = 5
    df = vae.z_to_smiles(x, decode_attempts=100, noise_norm=noise)
    smiles = {k : vae.smiles_to_hot(mu.canon_smiles(k)) for k in set(df['smiles'])}
    best_smile = None
    best_smile_score = -1
    for smile in smiles:
        score = score_fn(vae.predict_prop_Z(vae.encode(smile))[0])
        if score > best_smile_score:
            best_smile = smile
            best_smile_score = score
    return best_smile_score

In [ ]:
os.chdir(cur_dir)
vae = vae_utils.VAEUtils(directory=directory)